In [1]:
import pandas as pd
import numpy as np

In [2]:
data=pd.read_csv('Downloads/IMDB dataset.csv')

In [3]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
data.shape

(50000, 2)

In [5]:
data['sentiment'].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

In [6]:
data.isnull().sum()

review       0
sentiment    0
dtype: int64

In [6]:
 data.columns

Index(['review', 'sentiment'], dtype='object')

In [7]:
type(data)

pandas.core.frame.DataFrame

In [8]:
type(data.sentiment)

pandas.core.series.Series

In [9]:
data.isnull().any()

review       False
sentiment    False
dtype: bool

In [10]:
data.describe(include='all')

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [11]:
import nltk

In [12]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Text Normalization

In [16]:
import seaborn as sns

In [15]:
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import wordcloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [17]:

import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob,Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from bs4 import BeautifulSoup

In [20]:
# Tokenization of text
tokenizers=ToktokTokenizer()
#setting English  stopwords
stopwords=nltk.corpus.stopwords.words('english')
stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [21]:
def noiseremoval_text(text):
    soup=BeautifulSoup(text,'html.parser')
    text=soup.get_text()
    text=re.sub('\[[^]]*\]','',text)
    return text

In [22]:
# apply function on review column
data['review']=data['review'].apply(noiseremoval_text)

# Stemming

In [24]:
#stemming the text
def stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text=' '.join([ps.stem(word) for word in text.split()])
    return text

In [25]:
# apply function on review column
data['review']=data['review'].apply(stemmer)

In [55]:
data['review'].head()

0    one of the other review ha mention that after ...
Name: review, dtype: object


# removing stopwords

In [28]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [29]:
#set stopwords to english
stop_wr=set(stopwords.words('english'))

In [43]:
# removing the stop words
def removing_stopwords(text,is_lower_case=False):
    # tokenization of the text
    tokenizers=ToktokTokenizer()
    #setting the english words
    tokens=tokenizers.tokenize(text)
    tokens=[i.strip() for i in tokens]
    if is_lower_case:
        filtokens=[i for i in tokens if i not in stop_wr]
    else:
        filtokens=[i for i in tokens if i.lower() not in stop_wr]
    filtered_texts=' '.join(filtokens)
    return filtered_texts

In [44]:
# apply function on review column
data['review']=data['review'].apply(removing_stopwords)

In [45]:
data.head()

,review,sentiment
0,one review ha mention watch 1 oz episod ' hook...,positive
1,wonder littl production. film techniqu veri un...,positive
2,thought thi wa wonder way spend time hot summe...,positive
3,basic ' famili littl boy ( jake ) think ' zomb...,negative
4,"petter mattei ' "" love time money "" visual stu...",positive


In [46]:
train_review=data['review'][:30000]
test_review=data['review'][30000:]

# Bag of words

In [48]:
# count vectorizer for bag of words
cv=CountVectorizer(min_df=0,max_df=1,binary=False,ngram_range=(1,3))
# transform train reviews
cv_train=cv.fit_transform(train_review)
# transform test revies
cv_test=cv.transform(test_review)
print('Bow_cv_train : ',cv_train.shape)
print('Bow_cv_test : ',cv_test.shape)


Bow_cv_train :  (30000, 4954417)
Bow_cv_test :  (20000, 4954417)


# TF_IDF

In [49]:
# tfidf vectorizer

In [51]:
tf=TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,3))

In [52]:
# transformed train reviews
tf_train=tf.fit_transform(x_train)
# transformed test reviews
tf_test=tf.transform(x_test)
print('tfidf_train : ',tf_train.shape)
print('tfidf_test : ',tf_test.shape)


tfidf_train :  (30000, 4954417)
tfidf_test :  (20000, 4954417)


# label encoding 

In [56]:
# labeling the sentiment data
label=LabelBinarizer()
# transformed sentiment data
sentiment_data=label.fit_transform(data['sentiment'])
print(sentiment_data.shape)

(50000, 1)


In [60]:
train_data=data.sentiment[:30000]
test_data=data.sentiment[30000:]

In [61]:
# training the model 
logistic=LogisticRegression(penalty='l2',max_iter=500,C=1,random_state=42)
#fitting the model for bag words
lr_bow=logistic.fit(cv_train,train_data)
print(lr_bow)
# fitting the model for tfidf features
lr_tfidf=logistic.fit(tf_train,train_data)
print(lr_tfidf)

LogisticRegression(C=1, max_iter=500, random_state=42)
LogisticRegression(C=1, max_iter=500, random_state=42)


In [62]:
# predictig the model for bag of words
lr_bow_predict=logistic.predict(cv_test)
print(lr_bow_predict)

['negative' 'negative' 'negative' ... 'negative' 'positive' 'positive']


In [63]:
lr_tfidf_predict=logistic.predict(tf_test)
print(lr_tfidf_predict)

['negative' 'negative' 'negative' ... 'negative' 'positive' 'positive']


In [65]:
# accuracy score for bag of words
lr_bow_score=accuracy_score(test_data,lr_bow_predict)
print(lr_bow_score)

0.7425


In [66]:
# accuracy score for tfidf features
lr_tfidf_score=accuracy_score(test_data,lr_tfidf_predict)
print(lr_tfidf_score)

0.74265


# completed